In [288]:
import pandas as pd
url = 'https://raw.githubusercontent.com/ohhappylife/ETLCFA/master/merged_news(keyword%3D\'elizabeth\').csv'
df = pd.read_csv(url,index_col=0)
df['Author'] = df['Author'].astype(str).str.lstrip()

In [289]:
information = df.groupby('camefrom')['camefrom'].count()

In [290]:
import numpy as np
information = information.to_frame().reset_index(drop = True)
information = information.join(df.groupby('camefrom')['Author'].nunique().to_frame().reset_index(drop = True))
information = information.join(df.groupby('camefrom')['Title_without_stopwords'].nunique().to_frame().reset_index(drop = True))
information['%_duplicated'] = ((information.iloc[:, 1] /  information.iloc[:, 0])) * 100
df['length'] = df['Title_without_stopwords'].apply(len)
#information = information.join(df.groupby('camefrom')['length'].mean().reset_index(drop = True))
information = information.join(df.groupby('camefrom')['length'].agg([np.std, np.min, np.max, np.average]).reset_index(drop = True)
)

information.rename(columns={'Author': 'count_sources',
                            'camefrom': 'count_articles', 
                            'Title_without_stopwords' : 'count_duplicated_articles',
                            '%_duplicated' : '%_duplicated_articles',
                            'std' : 'std_article_count',
                            'amin' : 'min_article_count',
                            'amax' : 'max_article_count',
                            'average' : 'average_article_count'
                             }, inplace=True)



In [291]:
text = df.groupby('camefrom')['camefrom'].count()
text = text.to_frame().reset_index(drop = True)
text = text.join(df.groupby('camefrom')['Text_without_stopwords'].nunique().to_frame().reset_index(drop = True))
text['%_duplicated'] = ((text.iloc[:, 1] /  text.iloc[:, 0])) * 100
df['length'] = df['Text_without_stopwords'].apply(len)
#information = information.join(df.groupby('camefrom')['length'].mean().reset_index(drop = True))
text = text.join(df.groupby('camefrom')['length'].agg([np.std, np.min, np.max, np.average]).reset_index(drop = True)
)

text.rename(columns={'Author': 'count_sources',
                     'camefrom': 'count_texts', 
                     'Text_without_stopwords' : 'count_duplicated_texts',
                     '%_duplicated' : '%_duplicated_texts',
                     'std' : 'std_texts_count',
                     'amin' : 'min_texts_count',
                     'amax' : 'max_texts_count',
                     'average' : 'average_texts_count'
                     }, inplace=True)

In [292]:
df2 = df.drop_duplicates(subset = 'Text_without_stopwords')
df2 = df2.drop_duplicates(subset = 'Title_without_stopwords')
df2.groupby('Author')['Author'].count().sort_values(ascending = False).head(15)

Author
The New York Times              44
The Guardian                    31
Reuters                         19
Time                            14
USA Today                        8
Yahoo Entertainment              5
ABC News                         5
International New York Times     4
Business Insider                 4
NPR                              3
Fox News                         2
Torstar Open Data Team           2
GMA                              2
Global News                      2
AP                               2
Name: Author, dtype: int64

In [293]:
merged = information.join(text)

,count_articles,count_sources,count_duplicated_articles,%_duplicated_articles,std_article_count,min_article_count,max_article_count,average_article_count,count_texts,count_duplicated_texts,%_duplicated_texts,std_texts_count,min_texts_count,max_texts_count,average_texts_count
0,10,8,10,80.0,22.238355,39,104,69.100000,10,10,100.000000,26.880187,112,188,149.100000
1,7,7,7,100.0,19.082528,23,84,60.857143,7,7,100.000000,13.021959,85,124,109.285714
2,50,4,50,8.0,14.504946,20,77,49.120000,50,50,100.000000,30.328736,50,166,107.620000
3,100,19,100,19.0,16.556225,26,123,63.550000,100,99,99.000000,13.263316,136,208,174.940000
4,50,46,14,92.0,10.169502,49,90,57.360000,50,34,68.000000,151.248648,384,813,719.360000
5,30,21,30,70.0,36.455626,35,211,80.233333,30,1,3.333333,0.000000,11,11,11.000000
